# Install Syncthing

* Purpose: synchronize directories between computers on local network
    * I use this to synchronize folders between my Mac and the Pi
* [Official Site](https://syncthing.net/)
* [Install Guide](https://pimylifeup.com/raspberry-pi-syncthing/)

## Install Software

In [ ]:
%%bash

sudo apt update
sudo apt -y full-upgrade
sudo apt -y install apt-transport-https
curl -s https://syncthing.net/release-key.txt | sudo apt-key add -
echo "deb https://apt.syncthing.net/ syncthing stable" | sudo tee /etc/apt/sources.list.d/syncthing.list
sudo apt update
sudo apt install syncthing

## Configure software

In [ ]:
%%bash

# list of filed/dirs that never sync (https://docs.syncthing.net/users/ignoring.html)
cat << 'EOF' > ~/.stignore
(?d).DS_Store
.envrc
.direnv
.git
.ipynb_checkpoints*
__pycache__
ehthumbs.db
Icon?
Thumbs.db

EOF

# start then kill to create default configuration
syncthing &
last_pid=$!
echo $last_pid
sleep 5
kill -KILL $last_pid

# web access from any authorized device
cd ~/.config/syncthing
cp config.xml config.xml.bak
sed -i 's/127\.0\.0\.1:8384/0\.0\.0\.0:8384/g' config.xml

# install service
sudo tee /lib/systemd/system/syncthing.service << EOF
[Unit]
Description=Syncthing - Open Source Continuous File Synchronization
Documentation=man:syncthing(1)
After=network.target

[Service]
User=pi
ExecStart=/usr/bin/syncthing -no-browser -no-restart -logflags=0
Restart=on-failure
RestartSec=5
SuccessExitStatus=3 4
RestartForceExitStatus=3 4

# Hardening
ProtectSystem=full
PrivateTmp=true
SystemCallArchitectures=native
MemoryDenyWriteExecute=true
NoNewPrivileges=true

[Install]
WantedBy=multi-user.target
EOF

# start service
sudo systemctl enable syncthing
sudo systemctl start syncthing

## Verify Syncthing is running

In [1]:
!sudo systemctl --no-pager status syncthing

Password:


## Configure from Web Interface

Consult the [Syncthing documentation](https://syncthing.net/) or the [Install Guide](https://pimylifeup.com/raspberry-pi-syncthing/) for details.

Summary of procedure:

* go to http://yourpi.local:8384
* Click `Danger/Settings`
   * Select `GUI` tab
   * Enter GUI Authentication user and password
      * e.g. your name and something that's hard to guess
   * click Save
* Click `Actions/Show ID`
   * Copy ID to Syncthing on remote (e.g. Mac) you want to share files with
      * Note: Syncthing must be running on remote!
      * on your remote, go to http://localhost:8384
      * `Add Remote Device`
      * On Pi, accept remote request
      * On remote (Mac), the new Pi will show up
      * Click on it and press `Edit`
      * Add the folders you want to share